<a href="https://colab.research.google.com/github/gunjanak/Data_Mining/blob/main/Movie_review_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Train the model with imdb data. Read a movive review from metacritic and label it positive or negative***

#Part I : Training the model

In [79]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Text Mining/Movie_review/IMDB Dataset.csv')

In [81]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [82]:
df['Positively Rated'] = np.where(df['sentiment']=='positive',1,0)

In [83]:
df.head()

,review,sentiment,Positively Rated
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [84]:
len(df[df['Positively Rated'] == 1])

25000

In [85]:
len(df)

50000

df['review'] is X


df['Positively Rated'] is y

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(df['review'],df['Positively Rated'],random_state=0)

In [87]:
print('X_train first entry:\n\n',X_train[0])
print('\n\nX_train_shape: ',X_train.shape)

X_train first entry:

 One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the

The bag-of-words approach is simple and commonly used way to represent text for
use in machine learning, which ignores structure and
only counts how often each word occurs.
CountVectorizer allows us to use the bag-of-words approach
by converting a collection of text documents into a matrix of token counts. 

In [88]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(X_train)

In [89]:
len(vect.get_feature_names())

90506

we use the transform method to transform the documents in X_train to
a document term matrix, giving us the bag-of-word representation of X_train.
This representation is stored in a SciPy sparse matrix, where each row corresponds
to a document and each column a word from our training vocabulary.
The entries in this matrix
are the number of times each word appears in each document.
Because the number of words in the vocabulary is so
much larger than the number of words that might appear in a single review,
most entries of this matrix are zero. 

In [91]:
X_train_vectorized = vect.transform(X_train)

Training Model

In [92]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
model1.fit(X_train_vectorized,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [93]:
X_test.shape

(12500,)

In [94]:
from sklearn.metrics import roc_auc_score
predictions = model1.predict(vect.transform(X_test))
print('AUC: ',roc_auc_score(y_test,predictions))

AUC:  0.8822758683068064


In [95]:
print(model1.predict(vect.transform(['not an issue, phone is working','an issue, phone is not working'])))

[0 0]


One way we can add some context is by adding sequences of word features
known as n-grams.
For example, bigrams, which count pairs of adjacent words,
could give us features such as is working versus not working.
And trigrams, which give us triplets of adjacent words,
could give us features such as not an issue.
To create these n-gram features,
we'll pass in a tuple to the parameter ngram_range,
where the values correspond to the minimum length and maximum lengths of sequences.
For example, if I pass in the tuple, 1, 2, CountVectorizer
will create features using the individual words, as well as the bigrams. 

In [96]:
#using only word of frequency 5
#using individual word, bigrams and trigrams
vect1 = CountVectorizer(min_df=5,ngram_range=(1,3)).fit(X_train)
X_train_vectorized1 = vect1.transform(X_train)
len(vect1.get_feature_names())

366814

In [97]:
model2 = LogisticRegression()
model2.fit(X_train_vectorized1,y_train)
predictions = model2.predict(vect1.transform(X_test))
print('AUC: ',roc_auc_score(y_test,predictions))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AUC:  0.9046495415265102


In [99]:
print(model2.predict(vect1.transform(['not an issue, phone is working','an issue, phone is not working'])))

[1 0]


In [100]:
print(model1.predict(vect.transform(['not an issue, phone is working','an issue, phone is not working'])))

[0 0]


#Part II: Scrapping the review

Ask user to enter a movie

Scrape reviews of that movie from metacritic and put it in database

In [124]:
!pip3 install requests_html bs4

In [125]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from urllib.parse import urljoin

In [126]:
session = HTMLSession()

In [127]:
def get_all_forms(url):
    """Returns all form tags found on a web page's `url` """
    # GET request
    res = session.get(url)
    # for javascript driven website
    # res.html.render()
    soup = BeautifulSoup(res.html.html, "html.parser")
    return soup.find_all("form")

In [128]:
def get_form_details(form):
    """Returns the HTML details of a form,
    including action, method and list of form controls (inputs, etc)"""
    details = {}
    # get the form action (requested URL)
    action = form.attrs.get("action").lower()
    # get the form method (POST, GET, DELETE, etc)
    # if not specified, GET is the default in HTML
    method = form.attrs.get("method", "get").lower()
    # get all form inputs
    inputs = []
    for input_tag in form.find_all("input"):
        # get type of input form control
        input_type = input_tag.attrs.get("type", "text")
        # get name attribute
        input_name = input_tag.attrs.get("name")
        # get the default value of that input tag
        input_value =input_tag.attrs.get("value", "")
        # add everything to that list
        inputs.append({"type": input_type, "name": input_name, "value": input_value})
    # put everything to the resulting dictionary
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details

In [129]:
url = "https://www.metacritic.com/"
# get all form tags
forms = get_all_forms(url)
# iteratte over forms
for i, form in enumerate(forms, start=1):
    form_details = get_form_details(form)
    print("="*50, f"form #{i}", "="*50)
    print(form_details)

================================================== form #1 ==================================================
{'action': '/search', 'method': 'post', 'inputs': [{'type': 'text', 'name': 'search_term', 'value': ''}, {'type': 'hidden', 'name': 'search_filter', 'value': 'all'}]}


In [130]:
# get the first form
first_form = get_all_forms(url)[0]

In [131]:
movie = input('Enter name of movie: ')

Enter name of movie: bride wars


In [132]:
# the data body we want to submit
data = {}
for input_tag in form_details["inputs"]:
    if input_tag["type"] == "hidden":
        # if it's hidden, use the default value
        data[input_tag["name"]] = input_tag["value"]
    elif input_tag["type"] != "submit":
        # all others except submit, prompt the user to set it
        #value = input(f"Enter the value of the field '{input_tag['name']}' (type: {input_tag['type']}): ")
        value = movie
        data[input_tag["name"]] = value

In [133]:
# join the url with the action (form request URL)
url = urljoin(url, form_details["action"])

if form_details["method"] == "post":
    res = session.post(url, data=data)
elif form_details["method"] == "get":
    res = session.get(url, params=data)

In [134]:
import re

In [135]:
soup = BeautifulSoup(res.text, 'html.parser')
#a = soup.find_all("h3")
a = soup.find_all("h3",{"class": "product_title basic_stat"}) 
result = []
for th in a:
    #print(th)
    result.append(th)
b = str(result[0])
cc = re.findall(r'"([^"]*)"', b)
d = cc[1]
url = 'https://www.metacritic.com'+d
print(url)

https://www.metacritic.com/movie/bride-wars


In [136]:
import requests

In [137]:
def reviews(url):
  user_agent = {'User-agent': 'Mozilla/5.0'}
  response  = requests.get(url, headers = user_agent)
  soup = BeautifulSoup(response.text, 'html.parser')
  a = soup.find_all("div",{"class": "review_body"}) 
  reviews = []
  for i in a:
    reviews.append(str(i))

  all_reviews = []
  for review in reviews:
    zz = re.findall(r'<span[^>]*>[\s\S]+<\/span>',review)
    zzz = zz[0]
    zzzz = zzz[:-7]
    zzzzz = zzzz[6:]
    output = re.findall(r'(>(.*?)<)',zzzzz)
    if not output:
      all_reviews.append(zzzzz)
    else:
      all_reviews.append(output[1][1])
    #print(output[1])
    #print(output[1][1])
  return all_reviews

In [138]:
all_reviews = reviews(url)

In [139]:
all_reviews

['Simplemente Anne Hathaway es una gran actriz, sus películas son muy amadas en México, en especial está película, no entiendo porque tiene mala crítica.',
 'Bride wars was directed by Gary Winick and has been released in may 2009. In this comedy there are several famous actors such as Anne Hathaway, Kate Hudson and Bryan Greenberg.\r',
 'The movie was fairly entertaining but a little on the goofy side and with many parts unbelievable. Hathaway had drawn my attention into this one - and other than that the story was mildly interesting even though a bit ludicrous (not in a good way). I still have to give it above average',
 "Two bi-polar woman with far too much money and time on their hands seek to destroy each others weddings through a series of photo montages and Candice Bergen's blathering narration.",
 "'Bride Wars' is a lousy, unfunny, sexist and uneven comedy that takes its two talented leads with a potentially funny premise, and wastes it in every way possible, leading to a very 

In [140]:
len(all_reviews)

7

***Predicting sentiment of reviews with both models and put result  in dataframe***

In [141]:
df = pd.DataFrame(columns=['Movie', 'Review','Rating_Model1','Rating_Model2'])

In [142]:
df.head()

,Movie,Review,Rating_Model1,Rating_Model2


In [143]:

for review in all_reviews:
  rating1 = model1.predict(vect.transform([review]))
  rating2 = model2.predict(vect1.transform([review]))
  df = df.append({'Movie':movie,'Review':review,'Rating_Model1':rating1,'Rating_Model2':rating2},ignore_index = True) 



In [144]:
df

,Movie,Review,Rating_Model1,Rating_Model2
0,bride wars,"Simplemente Anne Hathaway es una gran actriz, ...",[1],[1]
1,bride wars,Bride wars was directed by Gary Winick and has...,[0],[1]
2,bride wars,The movie was fairly entertaining but a little...,[0],[0]
3,bride wars,Two bi-polar woman with far too much money and...,[0],[0]
4,bride wars,"'Bride Wars' is a lousy, unfunny, sexist and u...",[0],[0]
5,bride wars,Regarding the three weddings I served as best...,[1],[0]
6,bride wars,"The worst movie of 2009, so far. And I have a...",[0],[0]
